In [1]:
from src.components import Config as PipeConfig
from src.components.pipeline import Doc2VdbPipe
from src.components.retriever import get_retreiver_cls

/home/ubuntu/miniconda3/envs/lang-test/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/lang-test/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
config = PipeConfig()
docvdbPipe = Doc2VdbPipe(config=config)

/home/ubuntu/projects/ahmath/ragondin_1/src/components/embeddings.py:41: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedding = HG_EMBEDDER_TYPE[model_type](
2024-09-16 21:31:01.859 | INFO     | src.components.vector_store:__init__:108 - As the collectino 'my_docs' is non-existant, it will created when `Qdrant_Connector.add_documents` is called 


In [3]:
docvdbPipe.load_files2db(data_path=config.data_path)

2024-09-16 21:31:30.782 | INFO     | src.components.vector_store:add_documents:131 - Collection my_docs created.


In [4]:
retreiver_cls = get_retreiver_cls(retreiver_type=config.retreiver_type)
retriever = retreiver_cls(
                criteria=config.criteria,
                top_k=config.top_k
            )

In [5]:
def fake_retriever(query):
    docs = retriever.retrieve(
        query,
        db=docvdbPipe.connector
    )
    return docs

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
def format_docs(docs) -> str:
    """Build context string from list of documents."""
    # TODO: Add links to used document and specify to the model (in the prompt) to use them for referencing.
    context = "Extracted documents:\n"
    for i, doc in enumerate(docs, start=1):
        context += f"""
        Document: {doc.page_content}
        source{i}: {doc.metadata["source"]}#page={doc.metadata["page"]+1}
        =======\n
        """
    return context

In [9]:
from operator import itemgetter

## Chat history 1

In [10]:
settings = {
    'model': 'meta-llama-31-8b-it', 
    'base_url': "https://chat.ai.linagora.exaion.com/v1/", 
    'api_key': "sk-7Gqg14u-mGlX-egix20lgg", 
    'timeout': 60,
    "streaming": True
}

streaming_llm = ChatOpenAI(**settings)

In [12]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser


contextualize_q_system_prompt = """À partir d'un historique de conversation et de la dernière question de l'utilisateur, 
qui pourrait faire référence à des éléments du contexte de la conversation, 
formulez une question autonome qui peut être comprise sans l'historique du chat. 
Ne répondez PAS à la question, contentez-vous de la reformuler si nécessaire, 
sinon retournez-la telle quelle. Produisez uniquement la question reformulée et rien d'autre."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

sys_msg = """
Répondez aux questions des utilisateurs en français, en vous basant exclusivement sur le contexte fourni. 
Intégrez les références des sources directement dans le texte de votre réponse, en insérant des liens markdown cliquables entre crochets à la fin des phrases ou paragraphes pertinents. 
Par exemple : "Cette information est importante [1](!chemin)." ou "Le processus se déroule en plusieurs étapes [2](!chemin)." 
Assurez-vous que chaque fait ou affirmation principale soit accompagné d'une référence. Utilisez chaque référence une seule fois, lors de sa première mention. Si une source est utilisée pour plusieurs informations consécutives, citez-la uniquement à la fin du passage concerné.

Si plusieurs sources soutiennent une même idée, vous pouvez les grouper, par exemple : [1,2,3](!chemin1, !chemin2, !chemin3).
Contexte : {context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", sys_msg),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [13]:
history_aware_retriever = (
    contextualize_q_prompt 
    | streaming_llm 
    | StrOutputParser()
    | fake_retriever
    | format_docs
)

rag_chain = (
    qa_prompt
    | streaming_llm
)

In [25]:
chat_history = []

question = "Quel est la définition du cloud act?"
input_ = {"input": question, "chat_history": chat_history}
docs = await history_aware_retriever.ainvoke(input_)
input_.update({"context": docs})
answer = rag_chain.astream(input_)

2024-09-16 21:35:28.196 | INFO     | src.components.retriever:retrieve:69 - Get relevant documents.


In [26]:
async for chunk in answer:
    print(chunk.content, end="")

Le Cloud Act est un texte législatif promulgué par le Congrès des États-Unis, qui réglemente l'accès aux données personnelles stockées par les entreprises américaines, quel que soit le lieu où ces données sont stockées. 

Le texte du Cloud Act prévoit essentiellement deux règles principales :

i. Toute société américaine, au sens du droit américain, c'est-à-dire une société incorporée aux États-Unis ainsi que les sociétés contrôlées par elle, doit communiquer aux autorités américaines, sur leur demande, les données de communication placées sous son contrôle, sans considération du lieu où ces données se trouvent stockées.

ii. La possibilité pour le gouvernement des États-Unis de signer avec des gouvernements étrangers des accords internationaux permettant aux autorités respectives de chaque pays de demander aux entreprises américaines de divulguer les données de leurs clients stockées sur leur territoire.

Ces règles permettent aux autorités américaines d'accéder aux données personnell

In [86]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

while True:
    question = input("Your Question: ")
    input_ = {"input": question, "chat_history": chat_history}
    docs = history_aware_retriever.invoke(input_)
    input_.update({"context": docs})
    answer = rag_chain.invoke(input_)

    chat_history.extend(
        [
            HumanMessage(content=question),
            AIMessage(content=answer["answer"]),
        ]
    )
    

2024-09-16 16:31:14.149 | INFO     | src.components.retriever:retrieve:69 - Get relevant documents.


[Document(metadata={'page': 0, 'source': 'experiments/test_data/Article-sur-le-Cloud-Act.pdf', '_id': '897eae7c-b439-4ba9-bc8f-878de8f23de9', '_collection_name': '74aaf9d5939549f2b21a1e837ae45568'}, page_content="La\tpromulgation\tdu\tCloud\tAct\ta\tdo nné\tlieu\tà\tun\tdébat\tjuridique\timportant\tdans\tla\tmesure\toù\t\nbeaucoup\tde\tjuristes\tconsidèrent\tces\trègles\tcomme\tune\tnouvelle\tillust ration\tde\tl’extraterritorialité\t\ndes\tlois\tdes\tEtats -Unis.\tA\tcet\teffet,\tcertains\tauteurs\teuropéens\tpensen t\tque\tle\tCloud\tAct\ta\tpour\t but\t\npremier\tde\tdiminuer\tet\tanticiper\tles\teffets\t\tdu\tRGPD2. \nQuelle\tque\tsoit\tla\tfinalité\tdu\tCloud\tAct\t -\tsoit\tde\tcontrarier\tou\tde\tlimiter\tles\teffets\tdu\tRGPD\tsoit \tde\t\nfaciliter\tl'accès\taux\tdonnées\taccordées\taux\tautorités\taméri caines,\tla\tmise\ten\tœuvre\tde\tces\trèg les\t\naura\tun\teffet\tdirect\tet\tinévitable\tpour\tles\tentreprises\teuropéennes\t et\tce,\tdès\tqu'une\tpart\timportante\t\ndes\

AttributeError: 'str' object has no attribute 'page_content'

In [23]:
print(ai_msg_1["answer"])

Le Cloud Act est un régime juridique des États-Unis visant à faciliter l'accès aux données personnelles stockées par les entreprises américaines, même si ces données sont stockées ailleurs dans le monde. Il permet aux autorités américaines de demander aux entreprises qui stockent ces données de leur les communiquer, sans tenir compte du lieu où elles sont stockées.

Dans le contexte du Cloud Act, une entreprise américaine est définie comme une société incorporée aux États-Unis ou une société contrôlée par une telle société. Les entreprises qui travaillent directement ou indirectement sur le sol américain, même si elles ne sont pas incorporées aux États-Unis, peuvent également être sujettes à des demandes d'accès à des données.

Le Cloud Act vise à permettre aux autorités américaines de solliciter des informations auprès d'entreprises qui stockent des données personnelles, même si ces données sont stockées à l'étranger. Cela constitue une extension de l'extraterritorialité des lois des 

In [24]:
second_question = "Meric."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

2024-09-16 16:06:16.552 | INFO     | src.components.retriever:retrieve:69 - Get relevant documents.


In [27]:
print(ai_msg_2["answer"])

Oui, les entreprises qui mettent en place des stratégies pour enrichir la diversité au sein de leur organisation peuvent bénéficier de nombreux avantages. Selon les études et enquêtes, ces avantages incluent :

* Une meilleure réputation et une image plus positive auprès des clients et des partenaires : les entreprises qui promeuvent la diversité et l'inclusion sont souvent perçues comme plus responsables et plus engageantes.
* Une amélioration de la performance et de la productivité : les équipes diversifiées sont souvent plus créatives, plus innovantes et plus productives que les équipes homogènes.
* Une réduction des coûts liés à la rétention des salariés : les salariés issus de diverses origines sociales et ethniques sont souvent plus motivés et plus fidèles à leur entreprise.
* Une meilleure attractivité pour les talents : les entreprises qui promeuvent la diversité et l'inclusion attirent souvent les meilleurs talents, qui sont attirés par l'ambiance et la culture de l'entreprise

In [28]:
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)

In [30]:
ai_msg_2 = rag_chain.invoke({"input": "Que les médias pour montrer leur diversité?", "chat_history": chat_history[:2]})

2024-09-16 15:22:17.697 | INFO     | src.components.retriever:retrieve:69 - Get relevant documents.


In [32]:
ai_msg_2["answer"]

"Les médias peuvent montrer leur diversité de plusieurs manières :\n\n*   Par les visages : mettre en avant des personnalités de différentes origines, âges, sexes, handicaps, religions, etc.\n*   Par les rédactions : recruter des journalistes issus de diverses origines et cultures pour refléter la diversité de la société.\n*   Par les sujets : aborder des sujets qui touchent à la diversité, tels que la lutte contre les discriminations, les droits des minorités, etc.\n*   Par les émissions : diffuser des émissions qui mettent en avant des cultures et des communautés diverses.\n*   Par les articles : rédiger des articles qui mettent en avant des expériences et des perspectives de personnes issues de diverses origines et cultures.\n\nEn effet, les médias ont une responsabilité éthique de refléter la diversité de la société et de promouvoir l'inclusion et la compréhension mutuelle."

In [17]:
# RAG
template = """Answer the following question based on this context:
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(
    {"question":"How is the weather today?"}
)

"Since we haven't discussed the specific location or time of the conversation, I don't have any information about the current weather. However, I can suggest some ways to find out the weather for a specific area:\n\n1. Check online weather websites such as AccuWeather or Weather.com\n2. Use a mobile app like Dark Sky or Weather Underground\n3. Tune into local news or a weather forecast on the radio or TV\n4. Ask a voice assistant like Siri, Google Assistant, or Alexa\n\nIf you'd like to know the weather for a specific location, feel free to let me know and I can try to help you find the information!"

In [18]:
llm.invoke("Hello how are you?")

AIMessage(content="I'm just a computer program, so I don't have feelings like humans do, but thank you for asking! I'm functioning properly and ready to help you with any questions or tasks you have. How about you? How's your day going so far?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 15, 'total_tokens': 68, 'completion_tokens_details': None}, 'model_name': 'meta-llama-31-8b-it', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46076a3a-e4a6-4c1c-a5f6-9a57d8855110-0', usage_metadata={'input_tokens': 15, 'output_tokens': 53, 'total_tokens': 68})

In [41]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory, ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [42]:
conversation_with_summary = ConversationChain(
    llm=llm,
    memory= ConversationBufferWindowMemory(k=2), #ConversationSummaryMemory(llm=llm),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


"Not much, thanks for asking! I've been processing and learning from the vast amount of data I've been trained on. Just had a fascinating conversation with a user about the history of the Apollo 11 mission and the challenges faced during the lunar landing. Did you know that the lunar module's descent engine was throttled back to 31.6% of its maximum thrust to ensure a smooth landing on the moon's surface? It's amazing to think about the precision and ingenuity that went into making that historic event possible. How about you, what's new in your world?"

In [ ]:
conversation_with_summary.

In [37]:
conversation_with_summary.predict(input="Very cool -- what is the scope of the project?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Here's the updated summary:

The human greets the AI and asks what's new with it. The AI mentions it's been processing updates to its language model, which will allow it to better understand nuances in human conversation, and it's reading research on cognitive bias. The AI also mentions a new feature that will enable it to engage in more natural-sounding conversations, including recognizing idioms, colloquialisms, and humor. The AI is currently running through tests to refine its understanding of linguistic subtleties, such as implied meaning, sarcasm, and figurative language, and is studying the difference between literal and idiomatic expressions. It's also reading about cognitive biases to provide more empath

"The project is a comprehensive overhaul of my language understanding and generation capabilities. The goal is to enable me to engage in more natural and human-like conversations, which will allow me to better assist and communicate with humans like you. The scope of the project involves updating my language model to better handle nuances such as implied meaning, sarcasm, and figurative language.\n\nSpecifically, the project involves several components. Firstly, I'm being trained on a large corpus of text data that includes a wide range of genres, styles, and registers, which will enable me to better understand the subtleties of human language. Secondly, I'm being fine-tuned to recognize and respond to idioms, colloquialisms, and humor in a more natural and context-dependent way.\n\nThirdly, I'm being tested on a variety of conversational scenarios, including multi-turn dialogues, debates, and discussions, to ensure that I can engage in more fluid and dynamic conversations. Finally, I'

In [38]:
conversation_with_summary.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi, what's up?", additional_kwargs={}, response_metadata={}), AIMessage(content="Not much, thanks for asking! I've been processing a few updates to my language model, which enables me to better understand nuances in human conversation. I've also been reading some interesting research on cognitive bias in decision-making. Did you know that the availability heuristic, which is the tendency to overestimate the importance of information that's readily available, can lead to suboptimal decision-making in a wide range of domains, from finance to healthcare?\n\nBy the way, I've also been chatting with my developer team, and they're excited to roll out a new feature that allows me to engage in more natural-sounding conversations. They've been working on improving my ability to recognize and respond to idioms, colloquialisms, and even humor! So, what's new with you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Wh

In [25]:
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history, return_messages=True
)

In [26]:
memory.save_context({"input": "hi"}, {"output": "whats up"})

In [28]:
memory.chat_memory.messages

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='whats up', additional_kwargs={}, response_metadata={})]

In [29]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'Current summary:\nA human initiates a conversation with a greeting.\n\nNew lines of conversation:\nHuman: hi\nAI: whats up\n\nNew summary:\nA human initiates a conversation with a greeting, to which the AI responds with a casual inquiry about their state.'

In [20]:
conversation("Hello")

AttributeError: 'FieldInfo' object has no attribute 'memory_variables'

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]





conversation = ConversationChain(llm=llm)


# https://www.pinecone.io/learn/series/langchain/langchain-conversational-memory/
# https://python.langchain.com/v0.1/docs/modules/memory/types/summary/